In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Data Preparation of Climate dataset
climate_data = pd.read_excel('Dataset/archive/Merged_climate_data.xlsx')
climate_data.columns = climate_data.columns.str.replace('-07', '')
climate_data

,code,name,1950,1951,1952,1953,1954,1955,1956,1957,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,SheetName
0,AGO,Angola,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,fd
1,AND,Andorra,12.52,13.23,10.87,12.00,12.39,10.18,13.64,12.21,...,11.38,11.73,9.75,9.27,10.27,10.99,9.85,10.87,9.34,fd
2,ASM,American Samoa (U.S.),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,fd
3,AGO,Angola,45.40,62.05,65.64,84.28,52.09,84.92,86.49,106.08,...,87.80,89.09,91.35,90.15,97.34,87.53,92.94,87.56,100.19,pr
4,AND,Andorra,79.64,111.16,93.81,77.02,87.54,83.76,86.19,85.32,...,86.50,115.51,110.31,86.19,83.41,91.09,124.84,95.41,104.33,pr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,ALB,Albania,86.20,90.25,111.80,74.71,100.59,124.79,115.01,98.20,...,113.45,115.90,116.14,95.40,127.12,96.53,125.68,105.76,95.45,pr
620,DZA,Algeria,5.15,7.24,4.92,6.25,5.42,6.07,5.13,8.44,...,6.15,5.65,6.11,5.88,5.24,4.86,7.42,5.96,4.23,pr
621,AFG,Afghanistan,10.00,10.92,11.05,11.35,11.10,11.94,11.69,9.83,...,11.59,12.83,12.31,12.95,13.74,13.53,13.72,12.96,12.31,tas
622,ALB,Albania,13.19,12.81,12.98,11.92,11.51,12.19,11.40,12.26,...,13.06,13.20,13.15,13.26,13.16,13.13,13.73,13.57,13.37,tas


In [3]:
# Unpivot the DataFrame
climate_data = pd.melt(climate_data, id_vars=['code', 'name', 'SheetName'], var_name='Year', value_name='Value')
climate_data

,code,name,SheetName,Year,Value
0,AGO,Angola,fd,1950,0.00
1,AND,Andorra,fd,1950,12.52
2,ASM,American Samoa (U.S.),fd,1950,0.00
3,AGO,Angola,pr,1950,45.40
4,AND,Andorra,pr,1950,79.64
...,...,...,...,...,...
44299,ALB,Albania,pr,2020,95.45
44300,DZA,Algeria,pr,2020,4.23
44301,AFG,Afghanistan,tas,2020,12.31
44302,ALB,Albania,tas,2020,13.37


In [4]:
climate_data.columns = ['Country_Code', 'Country', 'Climate Variables', 'Year', 'Value']

In [5]:
#Pivot the Climate Variables values as columns
climate_data = climate_data.pivot_table(index=['Country_Code', 'Country', 'Year'], columns='Climate Variables', values='Value')
climate_data.reset_index(inplace=True)
climate_data

Climate Variables,Country_Code,Country,Year,fd,pr,tas
0,ABW,Aruba (Neth.),1950,0.0,87.00,25.89
1,ABW,Aruba (Neth.),1951,0.0,50.20,26.56
2,ABW,Aruba (Neth.),1952,0.0,44.75,26.58
3,ABW,Aruba (Neth.),1953,0.0,44.66,26.59
4,ABW,Aruba (Neth.),1954,0.0,70.32,26.07
...,...,...,...,...,...,...
14763,ZWE,Zimbabwe,2016,0.0,59.72,22.36
14764,ZWE,Zimbabwe,2017,0.0,81.14,21.23
14765,ZWE,Zimbabwe,2018,0.0,56.38,21.93
14766,ZWE,Zimbabwe,2019,0.0,50.78,22.45


In [6]:
climate_data.rename(columns={'fd': 'No of frost days', 'pr': 'Precipitation', 'tas':'Avg temperature'}, inplace=True)

In [7]:
#Import World Bank Indicators data
world_bank_data = pd.read_csv('Dataset/archive/World_Bank_Indicators.csv')
world_bank_data.dropna(inplace = True)
world_bank_data.columns = [col.split(' [')[0] for col in world_bank_data.columns]
world_bank_data.head(5)

,Country Name,Country Code,Series Name,Series Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,AFG,"Use of IMF credit (DOD, current US$)",DT.DOD.DIMF.CD,..,..,..,..,..,..,...,375643168.3,334839124.6,300161714.8,276984771.4,284853905.2,274596496.6,276923268.4,628740557.2,1185327218,..
1,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,..,..,..,..,..,..,...,-1.44590842723846,-1.36474287509918,-1.3485871553421,-1.53404927253723,-1.52172565460205,-1.48977172374725,-1.40771043300629,-1.48053777217865,-1.14237689971924,..
2,Albania,ALB,"Use of IMF credit (DOD, current US$)",DT.DOD.DIMF.CD,..,..,..,..,..,..,...,96755468.2,150856823.2,243583720.9,387084198.2,488777110.1,470321203.4,447707759.2,627019436.4,733945608.9,..
3,Albania,ALB,Control of Corruption: Estimate,CC.EST,..,..,..,..,..,..,...,-0.750955939292908,-0.586141347885132,-0.520959854125977,-0.44898048043251,-0.458256989717484,-0.524145126342773,-0.543374061584473,-0.551931083202362,-0.5559241771698,..
4,Algeria,DZA,"Use of IMF credit (DOD, current US$)",DT.DOD.DIMF.CD,..,..,..,..,..,..,...,1845203857.4,1735936449.6,1660361592.5,1610758149.3,1706376276.2,1666425596.1,1656880871,1725708072,4306075776.8,..


In [8]:
world_bank_data.drop('Series Code', axis=1, inplace=True)

In [9]:
world_bank_data = pd.melt(world_bank_data, id_vars=['Country Name', 'Country Code', 'Series Name'], var_name='Year', value_name='Value')
world_bank_data.to_csv('World_Bank_Ind.csv', index=False)

In [10]:
# world_bank_data = world_bank_data.pivot_table(index=['Country Name', 'Country Code', 'Year'], columns='Series Name', values='Value')
# world_bank_data.reset_index(inplace=True)
# world_bank_data
world_bank_data

,Country Name,Country Code,Series Name,Year,Value
0,Afghanistan,AFG,"Use of IMF credit (DOD, current US$)",1960,..
1,Afghanistan,AFG,Control of Corruption: Estimate,1960,..
2,Albania,ALB,"Use of IMF credit (DOD, current US$)",1960,..
3,Albania,ALB,Control of Corruption: Estimate,1960,..
4,Algeria,DZA,"Use of IMF credit (DOD, current US$)",1960,..
...,...,...,...,...,...
33511,Sub-Saharan Africa (IDA & IBRD countries),TSS,Control of Corruption: Estimate,2022,..
33512,Upper middle income,UMC,"Use of IMF credit (DOD, current US$)",2022,..
33513,Upper middle income,UMC,Control of Corruption: Estimate,2022,..
33514,World,WLD,"Use of IMF credit (DOD, current US$)",2022,..


In [11]:
world_bank_data.replace('..', 0.0, inplace=True)

In [12]:
world_bank_data = world_bank_data.pivot_table(index=['Country Name', 'Country Code', 'Year'], columns='Series Name', values='Value')
world_bank_data.reset_index(inplace=True)
world_bank_data

Series Name,Country Name,Country Code,Year,Control of Corruption: Estimate,"Use of IMF credit (DOD, current US$)"
0,Afghanistan,AFG,1960,0.000000,0.000000e+00
1,Afghanistan,AFG,1961,0.000000,0.000000e+00
2,Afghanistan,AFG,1962,0.000000,0.000000e+00
3,Afghanistan,AFG,1963,0.000000,0.000000e+00
4,Afghanistan,AFG,1964,0.000000,0.000000e+00
...,...,...,...,...,...
16753,Zimbabwe,ZWE,2018,-1.227581,4.708956e+08
16754,Zimbabwe,ZWE,2019,-1.273280,4.681985e+08
16755,Zimbabwe,ZWE,2020,-1.289440,4.876475e+08
16756,Zimbabwe,ZWE,2021,-1.257897,1.422011e+09
